In [1]:
"""Specification Test: Income segmentation: High income.

Swiss Metro Case Study
Demand Modeling: 1.202
Spring 2019
Alexandra Berke

"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

pandas = pd.read_table("../Data/swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR	 = Beta('ASC_CAR',0,-20,20,0)
ASC_SBB	 = Beta('ASC_SBB',0,-10,10,1)
ASC_SM	 = Beta('ASC_SM',0,-20,20,0)
B_CAR_COST	 = Beta('B_CAR_COST',0,-10,10,0)
B_HE	 = Beta('B_HE',0,-10,10,0)
B_SM_COST	 = Beta('B_SM_COST',0,-10,10,0)
B_CAR_TIME	 = Beta('B_CAR_TIME',0,-10,10,0)
B_TRAIN_TIME	 = Beta('B_TRAIN_TIME',0,-10,10,0)
B_SM_TIME	 = Beta('B_SM_TIME',0,-10,10,0)
B_TRAIN_COST	 = Beta('B_TRAIN_COST',0,-10,10,0)
B_SENIOR	 = Beta('B_SENIOR',0,-10,10,0)
B_GA	 = Beta('B_GA',0,-10,80,0)
B_FIRST	 = Beta('B_FIRST',0,-10,80,0)

# Define here arithmetic expressions for name that are not directly 
# available from the data

SENIOR  = DefineVariable('SENIOR', AGE   ==  5,database )
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

#Utilities
Car_SP = ASC_CAR * one + B_CAR_TIME * CAR_TT + B_CAR_COST * CAR_CO + B_SENIOR * SENIOR
SBB_SP = ASC_SBB * one + B_TRAIN_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
        + B_GA * GA + B_FIRST * FIRST
SM_SP = ASC_SM * one + B_SM_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
        + B_SENIOR * SENIOR + B_FIRST * FIRST

V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Here we use the "biogeme" way for backward compatibility
exclude = (((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  )) + (  CHOICE   ==  0  ) + (  AGE == 6  ) + (  INCOME != 3 ))>0
database.remove(exclude)

# Estimation of the model
logprob = bioLogLogit(V,av,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "SpecTest_SM_high_income"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_CAR=	-0.151
ASC_SM=	-0.00693
B_CAR_COST=	-0.00583
B_CAR_TIME=	-0.0185
B_FIRST=	0.373
B_GA=	2.03
B_HE=	-0.00624
B_SENIOR=	-0.518
B_SM_COST=	-0.0111
B_SM_TIME=	-0.0173
B_TRAIN_COST=	-0.0293
B_TRAIN_TIME=	-0.0163
                 Value   Std err     t-test       p-value  Rob. Std err  \
ASC_CAR      -0.151094  0.318240  -0.474782  6.349424e-01      0.310011   
ASC_SM       -0.006926  0.278853  -0.024836  9.801856e-01      0.266767   
B_CAR_COST   -0.005825  0.001445  -4.030836  5.557874e-05      0.001522   
B_CAR_TIME   -0.018490  0.001221 -15.137858  0.000000e+00      0.001369   
B_FIRST       0.372906  0.124575   2.993418  2.758715e-03      0.123348   
B_GA          2.033578  0.468975   4.336220  1.449536e-05      0.483629   
B_HE         -0.006237  0.002222  -2.807426  4.993912e-03      0.002249   
B_SENIOR     -0.517888  0.414433  -1.249631  2.114345e-01      0.413810   
B_SM_COST    -0.011076  0.000836 -13.246201  0.000000e+00      0.000985   
B_SM_TIME    -0.017256  0.001376 -12